In [5]:
import os
import rasterio
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [6]:
# data dir path
data_dir = "../uhi-analysis/gee_exports"

# init lists for results
years = []
mean_ndvi = []
mean_lst = []


In [7]:
def extract_ndvi_lst_timeseries(data_dir):
    """Extract mean NDVI and LST from GeoTIFFs in a folder and return DataFrame."""
    
    def process_file(file_path):
        """Process individual GeoTIFF and return year, NDVI mean, and LST mean."""
        file_name = os.path.basename(file_path)
        year = int(file_name.split("_")[-1].replace(".tif", ""))
        print(f"🔍 Processing {file_name}")

        with rasterio.open(file_path) as src:
            image = src.read()
            ndvi_data = image[0, :, :]
            lst_data = image[1, :, :] if image.shape[0] > 1 else None

            ndvi_valid = ndvi_data[ndvi_data != src.nodata]
            ndvi_mean = np.mean(ndvi_valid)

            if lst_data is not None:
                lst_valid = lst_data[lst_data != src.nodata]
                lst_mean = np.mean(lst_valid)
            else:
                lst_mean = np.nan

        return year, ndvi_mean, lst_mean

    # Filter and process all valid files
    all_files = [f for f in sorted(os.listdir(data_dir)) 
                 if f.endswith(".tif") and "NDVI_LST_" in f]
    results = [process_file(os.path.join(data_dir, f)) for f in all_files]

    # Unpack results into DataFrame
    df = pd.DataFrame(results, columns=['Year', 'Mean_NDVI', 'Mean_LST'])
    df.sort_values(by='Year', inplace=True)
    
    print("\n📊 Extracted NDVI/LST Means:")
    print(df)

    # Optional: Save to CSV
    df.to_csv("NDVI_LST_Timeseries.csv", index=False)

    return df

In [8]:
ndvi_lst_df = extract_ndvi_lst_timeseries(data_dir)

🔍 Processing NDVI_LST_2013.tif
🔍 Processing NDVI_LST_2014.tif
🔍 Processing NDVI_LST_2015.tif
🔍 Processing NDVI_LST_2016.tif
🔍 Processing NDVI_LST_2017.tif
🔍 Processing NDVI_LST_2018.tif
🔍 Processing NDVI_LST_2019.tif
🔍 Processing NDVI_LST_2020.tif
🔍 Processing NDVI_LST_2021.tif
🔍 Processing NDVI_LST_2022.tif
🔍 Processing NDVI_LST_2023.tif
🔍 Processing NDVI_LST_2024.tif

📊 Extracted NDVI/LST Means:
    Year  Mean_NDVI    Mean_LST
0   2013   0.161491  312.918915
1   2014   0.213335  309.964661
2   2015   0.202855  310.279694
3   2016   0.173177  317.306915
4   2017   0.166237  312.090149
5   2018   0.185763  313.077942
6   2019   0.192487  298.960815
7   2020   0.145172  317.600769
8   2021   0.180933  304.367371
9   2022   0.127046  315.007263
10  2023   0.233329  305.560181
11  2024   0.135826  311.083008
